In [6]:
import urllib, json 
import requests
from bs4 import BeautifulSoup
import io
import os
import glob
import shutil
import re
from csv import DictReader
import time
import ffmpeg

In [148]:
# Get codes from the csv file
with open("paste_here.csv") as f:
    long_links = [row["id"] for row in DictReader(f)]

codes =[]
for ll in long_links:
    code = re.findall(r'wvideo=(.*?)"', ll)
    codes.append(code[0])

codes

['dhmuvqu5pb', 'uy74q37qsx', 'nxbx8bnteg', 'xg7niy4dv2']

In [160]:
video_names = []
bin_links = []
json_links =[]
video_namecheck =[]

In [161]:
# Get json_links
for code in codes:
    ts_rem = "https://fast.wistia.com/embed/medias/"+str(code)+".json"
    json_links.append(ts_rem)
print(json_links)

['https://fast.wistia.com/embed/medias/dhmuvqu5pb.json', 'https://fast.wistia.com/embed/medias/uy74q37qsx.json', 'https://fast.wistia.com/embed/medias/nxbx8bnteg.json', 'https://fast.wistia.com/embed/medias/xg7niy4dv2.json']


In [162]:
# Get bin_links and video_names
for json_link in json_links:
    with urllib.request.urlopen(json_link) as url:
        data = json.loads(url.read().decode())
        if data['media']['assets']['ext' == "mp4"]:
            try:
                bin_links.append(data['media']['assets']['height' == "1080"]['url'])
            except:
                bin_links.append(data['media']['assets']['height' == "720" or 'height' == "480"]['url'])
            video_names.append(data['media']['name'])

print("Current Total links:",len(bin_links))

Current Total links: 4


In [163]:
bin_links

['https://embed-ssl.wistia.com/deliveries/7afd65ca2ce9d356b0c4dbb7ae133c3e.bin',
 'https://embed-ssl.wistia.com/deliveries/eb691a2f6a20ac29b87c0622ec8b72f2.bin',
 'https://embed-ssl.wistia.com/deliveries/7a3a2d3dedcaad49abc7ac66e828d874.bin',
 'https://embed-ssl.wistia.com/deliveries/b24ec5cfe2b124d3dd2432d42c3dc2dd.bin']

In [164]:
video_names

['1.1-GC-Introducción a la hidrología.mp4',
 '1.2-GC-SIG en la gestión hidrológica.mp4',
 '2.1-GC-Cuenca Hidrográfica.mp4',
 '2.2-GC-Manejo de cuencas, una visión integral.mp4']

In [165]:
video_names_f = []
for i in video_names:
    i = i.replace(".", "_", 1)
    i = re.sub("-.*?-", "_", i)
    video_names_f.append(i)
    
video_names_f

['1_1_Introducción a la hidrología.mp4',
 '1_2_SIG en la gestión hidrológica.mp4',
 '2_1_Cuenca Hidrográfica.mp4',
 '2_2_Manejo de cuencas, una visión integral.mp4']

In [166]:
# Check files' names
format = ".mp4"
for i in video_names_f:
    if format in i:
        video_namecheck.append(i)
    else:
        i += ".mp4"
        video_namecheck.append(i)
        
video_namecheck

['1_1_Introducción a la hidrología.mp4',
 '1_2_SIG en la gestión hidrológica.mp4',
 '2_1_Cuenca Hidrográfica.mp4',
 '2_2_Manejo de cuencas, una visión integral.mp4']

In [167]:
# Download videos
n = 1
for i, j in zip(bin_links, video_namecheck):
    print("Downloading file "+str(n)+": " + str(j))
    (
        ffmpeg
        .input(str(i))
        .output(str(j), vcodec='copy', **{'loglevel': "panic"})
        .run()
    )
    #time.sleep(2)
    n +=1

print("Download Complete, total links:", n-1)

Download Complete, total links: 4


In [35]:
# Get course title and section titles
a = requests.get('https://mastergis.teachable.com/courses/1073393/lectures/22874415')
soup = BeautifulSoup(a.content, 'html.parser')

course_title = []
for tag in soup.find_all("h2"):
        course_title.append(tag.text.strip())

section_titles = []
for tag in soup.find_all("div"):
    try: 
        tag['data-course-id']
        section_titles.append(tag.text.strip())
    except: continue

section_titles_f = []       
for title in section_titles:
    if title.startswith("Proyecto"):
        continue
    else:
        section_titles_f.append(title)
        
section_titles_f
course_title[0]

'ArcGIS Completo'

In [36]:
# Create folders and move the respectives files
if os.path.isdir(f'{course_title[0]}') is False:
    for i in  range(len(section_titles_f)):
        i+=1
        for j in glob.glob(f"{i}_*"):
            if os.path.isdir(f'{course_title[0]}/{i}_{section_titles_f[i-1]}') is False:
                os.makedirs(f'{course_title[0]}/{i}_{section_titles_f[i-1]}')
                shutil.move(j, f'{course_title[0]}/{i}_{section_titles_f[i-1]}')
            else:
                shutil.move(j, f'{course_title[0]}/{i}_{section_titles_f[i-1]}')

print("Folders created")

# Falta

In [6]:
os.chdir('../')
os.chdir("angel_downloader")

In [127]:
import requests
from bs4 import BeautifulSoup

a = requests.get('https://mastergis.teachable.com/courses/1073393/lectures/22874415')
soup = BeautifulSoup(a.content, 'html.parser')

code_videos = []

for tag in soup.find_all("li"):
    try: code_videos.append(tag['data-lecture-id'])
    except: continue
code_videos

['22874415',
 '22879904',
 '22879906',
 '22874416',
 '22874417',
 '22874418',
 '22879994',
 '23541124',
 '23152592',
 '23152593',
 '23152594',
 '23152762',
 '23152761',
 '23152763',
 '23153633',
 '23153645',
 '23541197',
 '23153654',
 '23153616',
 '23153655',
 '23153656',
 '23153635',
 '23153639',
 '23153631',
 '23153626',
 '23153658',
 '23153638',
 '23153623',
 '23153630',
 '23153637',
 '23153648',
 '23153660',
 '23153644',
 '23233320',
 '23153634',
 '23233321',
 '23233319',
 '23153650',
 '23153659',
 '23153649',
 '23153643',
 '23233322',
 '23541222',
 '23231466',
 '23231460',
 '23231455',
 '23231469',
 '23231458',
 '23231463',
 '23231456',
 '23231465',
 '23231472',
 '23231461',
 '23231451',
 '23231471',
 '23231467',
 '23231457',
 '23231464',
 '23231453',
 '23231462',
 '23231452',
 '23231470',
 '23231459',
 '23231468',
 '23231475',
 '23231454',
 '23231476',
 '23541229',
 '23541393',
 '23156696',
 '23156672',
 '23156682',
 '23156681',
 '23156678',
 '23156661',
 '23156670',
 '23541417',

In [128]:
len(code_videos)

179

In [20]:
import requests
from bs4 import BeautifulSoup

a = requests.get('https://mastergis.teachable.com/courses/1073393/lectures/22874415')
soup = BeautifulSoup(a.content, 'lxml')
soup

<!DOCTYPE html>
<html>
<head>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:32,height:32/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="icon" type="image/png"/>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:72,height:72/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="apple-touch-icon" type="image/png"/>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:144,height:144/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="apple-touch-icon" type="image/png"/>
<link href="https://assets.teachablecdn.com/splash-images/teachable-apple-startup-image.png" rel="apple-touch-startup-image" type="image/png"/>
<link href="https://assets.teachablecdn.com/splash-images/teachable-apple-startup-image_2x.png" rel="apple-touch-startup-image" type="image/png"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta charset="utf-8"/>
<meta content="auth

In [136]:
import requests
url = 'https://mastergis.teachable.com/courses/1073393/lectures/22874415'
values = {'username': 'rsanchezc12@unc.edu.pe',
          'password': 'pN84TPYGE6U9D9Q82P9KW'}

a = requests.post(url, data=values)
soup = BeautifulSoup(a.content, 'lxml')
soup

<!DOCTYPE html>
<html>
<head>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:32,height:32/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="icon" type="image/png"/>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:72,height:72/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="apple-touch-icon" type="image/png"/>
<link href="https://process.fs.teachablecdn.com/ADNupMnWyR7kCWRvm76Laz/resize=width:144,height:144/https://www.filepicker.io/api/file/CVPqS8V5RsejqO8muRem" rel="apple-touch-icon" type="image/png"/>
<link href="https://assets.teachablecdn.com/splash-images/teachable-apple-startup-image.png" rel="apple-touch-startup-image" type="image/png"/>
<link href="https://assets.teachablecdn.com/splash-images/teachable-apple-startup-image_2x.png" rel="apple-touch-startup-image" type="image/png"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta charset="utf-8"/>
<meta content="auth

In [ ]:
# Get course title and section titles
a = requests.get('https://mastergis.teachable.com/courses/1073393/lectures/22874415')
soup = BeautifulSoup(a.content, 'html.parser')

course_title = []
for tag in soup.find_all("h2"):
        course_title.append(tag.text.strip())

section_titles = []
for tag in soup.find_all("div"):
    try: 
        tag['data-course-id']
        section_titles.append(tag.text.strip())
    except: continue

section_titles_f = []       
for title in section_titles:
    if title.startswith("Proyecto"):
        continue
    else:
        section_titles_f.append(title)
        
section_titles_f
course_title[0]

In [ ]:
# Create folders and move the respectives files
if os.path.isdir(f'{course_title[0]}') is False:
    for i in  range(len(section_titles_f)):
        i+=1
        for j in glob.glob(f"{i}_*"):
            if os.path.isdir(f'{course_title[0]}/{i}_{section_titles_f[i-1]}') is False:
                os.makedirs(f'{course_title[0]}/{i}_{section_titles_f[i-1]}')
                shutil.move(j, f'{course_title[0]}/{i}_{section_titles_f[i-1]}')
            else:
                shutil.move(j, f'{course_title[0]}/{i}_{section_titles_f[i-1]}')

print("Folders created")

In [4]:
ans = input("""Optional: Would you like to I create folders for your files?
(y/n)?

""")

Optional: Would you like to I create folders for your files?
(y/n)?

y


In [5]:
if ans == "y":
    link = input("""Please, give me the link of the MasterGIS website
For example: https://mastergis.teachable.com/courses/1073393/lectures/22874415
    
""")
    print("Ty, program finished")
else: print("Ty, program finished")

Please, give the link of the MasterGIS website
For example: https://mastergis.teachable.com/courses/1073393/lectures/22874415
    
asdas
